In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
pip install torch transformers faiss-cpu pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 47.5 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

In [6]:
# pretrained Sentence-BERT model
model = SentenceTransformer("paraphrase-MiniLM-L12-v2")

csv_path = "/kaggle/input/pcap-2019-dira-125910/dirA.125910-packets.csv"
df = pd.read_csv(csv_path, header=None, names=["timestamp", "src_ip", "dst_ip", "protocol", "size"])

# packet data to text format for BERT processing
df["packet_text"] = df["src_ip"] + " " + df["dst_ip"] + " " + df["protocol"] + " " + df["size"].astype(str)

# Generate embeddings
embeddings = model.encode(df["packet_text"].tolist(), convert_to_numpy=True)

# Normalize embeddings (cuz FAISS works better with normalized vectors)
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

# Creating FAISS index
dimension = embeddings.shape[1] # 384 is embedding size for MiniLM
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)  # all embeddings stored in FAISS

print(f"FAISS index contains {index.ntotal} embeddings.")

# Saving FAISS index
faiss.write_index(index, "packet_embeddings.index")

# original packet data with embeddings
df.to_csv("packet_metadata.csv", index=False)

print("FAISS index and metadata saved.")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-6-90dac8e1cb9c>:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, header=None, names=["timestamp", "src_ip", "dst_ip", "protocol", "size"])


Batches:   0%|          | 0/51663 [00:00<?, ?it/s]

FAISS index contains 1653188 embeddings.
FAISS index and metadata saved.


## Performance logging

In [10]:
import time
import faiss
import numpy as np
import pandas as pd
import psutil
from sentence_transformers import SentenceTransformer
import os

In [11]:
def measure(pid, func, *args, **kwargs):
    process = psutil.Process(pid)
    
    # Initial resource usage
    start_cpu = process.cpu_percent(interval=None)
    start_mem = process.memory_info().rss / (1024 ** 2)  # Convert to MB
    start_time = time.time()
    
    # Run function
    result = func(*args, **kwargs)
    
    # Final resource usage
    end_cpu = process.cpu_percent(interval=None)
    end_mem = process.memory_info().rss / (1024 ** 2)  # Convert to MB
    end_time = time.time()
    
    # Compute diff
    cpu_usage = end_cpu - start_cpu
    mem_usage = end_mem - start_mem
    execution_time = end_time - start_time

    print(f"Function: {func.__name__} | Time: {execution_time:.4f}s | CPU: {cpu_usage:.2f}% | Mem: {mem_usage:.2f}MB")
    return result

### 1. Queryng a new packet
Steps: 
1. Convert the query packet into text format.
2. Generate its BERT embedding.
3. Normalize the embedding (since FAISS works best with normalized vectors).
4. Search the FAISS index for the k-nearest neighbors.
5. Return the top-k results with their distances (lower = more similar).

In [12]:
def query_faiss(index, query_embedding, k=5):
    distances, indices = index.search(query_embedding, k)
    return indices

# Query FAISS index with a random packet
query_text = ["192.168.1.1 192.168.1.2 TCP 100"]
query_embedding = model.encode(query_text, convert_to_numpy=True)
query_embedding = query_embedding / np.linalg.norm(query_embedding, axis=1, keepdims=True)

pid = os.getpid()
query_result = measure(pid, query_faiss, index, query_embedding, 5)
print(f"Top 5 similar packet indices: {query_result}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Function: query_faiss | Time: 0.2331s | CPU: 98.70% | Mem: 0.25MB
Top 5 similar packet indices: [[ 767365  325967 1001693 1234941 1645756]]


### 2. Large Insertions
Steps: 
1. Convert 50+ new packets into embeddings.
2. Normalize them.
3. Add to the FAISS index.

In [13]:
def insert_data(index, new_embeddings):
    index.add(new_embeddings)
    print(f"Inserted {len(new_embeddings)} new vectors into FAISS index.")

# Simulate inserting 50 new packets at random
new_packet_texts = [f"192.168.1.{i} 192.168.1.{i+1} TCP {i*10}" for i in range(50)]
new_embeddings = model.encode(new_packet_texts, convert_to_numpy=True)
new_embeddings = new_embeddings / np.linalg.norm(new_embeddings, axis=1, keepdims=True)

pid = os.getpid()
measure(pid, insert_data, index, new_embeddings)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Inserted 50 new vectors into FAISS index.
Function: insert_data | Time: 2.3957s | CPU: 99.80% | Mem: 0.06MB


### 3. Large Deletions
**Problem!!!**
**FAISS does not support direct deletion of individual embeddings.**

Workaround:
1. Remove entries from the metadata CSV.
2. Rebuild the FAISS index without the deleted embeddings.

In [14]:
def delete_from_faiss(index, delete_indices, embeddings):
    #delete by rebuilding the index with the remaining embeddings.
    if not isinstance(delete_indices, np.ndarray):
        delete_indices = np.array(delete_indices)

    # Ensure delete_indices are valid
    delete_indices = delete_indices[delete_indices < embeddings.shape[0]]

    # Create a mask for filtering
    mask = np.ones(embeddings.shape[0], dtype=bool)
    mask[delete_indices] = False

    # Select embeddings that are not deleted
    new_embeddings = embeddings[mask]

    # Rebuild the FAISS index with remaining embeddings
    new_index = faiss.IndexFlatL2(new_embeddings.shape[1])
    new_index.add(new_embeddings)

    return new_index, new_embeddings  # Return updated index and embeddings

delete_indices = np.random.choice(embeddings.shape[0], 50, replace=False)

pid = os.getpid()
index, embeddings = measure(pid, delete_from_faiss, index, delete_indices, embeddings)
print(f"Deleted {len(delete_indices)} embeddings from FAISS index.")

Function: delete_from_faiss | Time: 3.1473s | CPU: 99.80% | Mem: 4843.23MB
Deleted 50 embeddings from FAISS index.


### 4. Large Updates
Similar to delete + insert.

Just remove old embeddings from FAISS, recompute new ones, and reinsert.

In [15]:
def update_faiss(index, update_indices, new_packet_texts, embeddings):
    # update by deleting specific indices and inserting new embeddings
    index, embeddings = delete_from_faiss(index, update_indices, embeddings)

    # Compute new embeddings
    new_embeddings = model.encode(new_packet_texts, convert_to_numpy=True)
    new_embeddings = new_embeddings / np.linalg.norm(new_embeddings, axis=1, keepdims=True)

    # Add new embeddings to FAISS
    index.add(new_embeddings)

    # Append new embeddings to the stored array
    embeddings = np.vstack([embeddings, new_embeddings])

    return index, embeddings  # Return updated index and embeddings

# Simulate updating 50 packets
update_indices = np.random.choice(embeddings.shape[0], 50, replace=False)
new_packet_texts = [f"10.0.0.{i} 10.0.0.{i+1} UDP {i*5}" for i in range(50)]

pid = os.getpid()
index, embeddings = measure(pid, update_faiss, index, update_indices, new_packet_texts, embeddings)
print(f"Updated {len(update_indices)} embeddings in FAISS index.")

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Function: update_faiss | Time: 6.4721s | CPU: 100.10% | Mem: 4843.18MB
Updated 50 embeddings in FAISS index.
